In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
file_path = '/content/drive/My Drive/Colab Notebooks/Research Techniques II/merged_dataset_v2.csv'

In [3]:
import pandas as pd

df = pd.read_csv(file_path, index_col = 0)
df.head()

,Unnamed: 0,text,labels
0,0,Bu tətbiq niyə nömrəyə kod göndərmirr,0
1,1,Biz evin qida məhsullarini bravodan alırıq çox...,1
2,2,Pisdir çox exlaqsizliq bas alıb gedir,0
3,3,Qulag asmiram amma merifetli qardasdi Allah ca...,1
4,4,m10 kimi rahat gözəl tətbiq yoxdur,1


In [4]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 re

In [7]:
!pip install numpy==1.26.4 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
from gensim.models.fasttext import load_facebook_model

fasttext_model = load_facebook_model(
    '/content/drive/MyDrive/Colab Notebooks/Research Techniques II/cc.az.300.bin'
)

In [5]:
df['tokens'] = df['text'].str.lower().str.split()

In [6]:
import numpy as np

lengths = df['tokens'].apply(len)
max_len = int(np.percentile(lengths, 95))  # %95'lik sınır
print(f"%95 cümle uzunluğu sınırı: {max_len}")

%95 cümle uzunluğu sınırı: 34


In [7]:
max_len = 40

In [8]:
import numpy as np

embedding_dim = 300  # FastText vektör boyutu
max_len = 50         # %95 sınırı göz önünde bulundurularak

def sentence_to_matrix(tokens, max_len=max_len):
    matrix = []
    for word in tokens[:max_len]:
        if word in fasttext_model.wv:
            matrix.append(fasttext_model.wv[word])
        else:
            matrix.append(np.zeros(embedding_dim))
    while len(matrix) < max_len:
        matrix.append(np.zeros(embedding_dim))
    return np.array(matrix)

In [9]:
X_data = np.array([sentence_to_matrix(tokens) for tokens in df['tokens']])
y_data = df['labels'].values

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Bidirectional, Dense, Dropout, Input

# Model tanımı
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),  # (timesteps, embedding_dim)
    Bidirectional(GRU(64, return_sequences=False, reset_after=True, recurrent_activation='sigmoid',
                      activation='tanh', use_bias=True, unroll=False)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.7196 - loss: 0.5273 - val_accuracy: 0.7775 - val_loss: 0.4609
Epoch 2/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.7858 - loss: 0.4479 - val_accuracy: 0.7874 - val_loss: 0.4381
Epoch 3/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7912 - loss: 0.4350 - val_accuracy: 0.7950 - val_loss: 0.4356
Epoch 4/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8029 - loss: 0.4210 - val_accuracy: 0.7997 - val_loss: 0.4267
Epoch 5/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8071 - loss: 0.4104 - val_accuracy: 0.8002 - val_loss: 0.4279
Epoch 6/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8126 - loss: 0.4049 - val_accuracy: 0.8024 - val_loss: 0.4270
Epoch 7/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.8199 - loss: 0.3902 - val_accuracy: 0.8029 - val_loss: 0.4225
Epoch 8/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.8213 - loss: 

In [13]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

348/348 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [14]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

[[3587  981]
 [1245 5322]]
              precision    recall  f1-score   support

           0     0.7423    0.7852    0.7632      4568
           1     0.8444    0.8104    0.8270      6567

    accuracy                         0.8001     11135
   macro avg     0.7934    0.7978    0.7951     11135
weighted avg     0.8025    0.8001    0.8008     11135

